In [1219]:
import numpy as np
from sys import maxsize
import random

*R* : Set of *m* resources represented as (id, cost?) tuples

In [1220]:
m = 2
# q -> quantity of resources
q = 3

# id, quantity_available, cost
R = [(i, np.random.randint(1, q+1), np.random.randint(0, 100)) for i in range(m)]
RQ = np.array(R)[:,1]
print(R)
print(RQ)

[(0, 2, 22), (1, 3, 25)]
[2 3]


*N* : Set of *n* agents as (id, balance?, priority?)

In [1221]:
n = 2

# id, quantity_owned, capacity, priority
N = [(i, 0, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]
N

[(0, 0, 84, 83), (1, 0, 2, 35)]

Zero Allocations

In [1222]:
def init_allocs(resources, agents):
    allocs = []
    for index, i in enumerate(agents):
        r_i = [0 for x in resources]
        allocs.append(r_i)
        
    return allocs

A = init_allocs(R, N)
print(A)

[[0, 0], [0, 0]]


Random allocations

In [1223]:
def rand_allocs(resources, agents):
    allocs = init_allocs(resources, agents)
    for index, i in enumerate(resources):
        res_remaining = q
        for jindex, j in enumerate(agents):
            if jindex == len(agents) - 1:
                allocs[jindex][index] = res_remaining
                res_remaining = 0
            else:
                n_res = np.random.randint(0, res_remaining+1)
                allocs[jindex][index] = n_res
                res_remaining -= n_res
    return allocs

In [1224]:
A = rand_allocs(R, N)
print(A)

[[2, 2], [1, 1]]


Random utility values of resources for both users : U

In [1225]:
U = []

for index, i in enumerate(N):
    u_i = random.sample(range(0, 100), len(R))
    U.append(u_i)
    
    
print(U)

[[11, 25], [29, 1]]


Random pair-wise complementing utility values : V

In [1226]:
V = []

for index, i in enumerate(N):
    v_ijjp1 = random.sample(range(0, 100), len(R)//2)
    V.append(v_ijjp1)
    
    
print(V)

[[27], [81]]


In [1227]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        if alloc[i] == 0 or alloc[i+1] == 0:
            pairwise_alloc.append(0)
        else:
            pairwise_alloc.append(alloc[i] if alloc[i] < alloc[i+1] else alloc[i+1])
        
#     print(alloc)
#     print(pairwise_alloc)
#     print(utility)
#     print(p_utility)
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

In [1228]:
print(additive_utililty(A[0], U[0]))
print(complementing_utility(A[0], U[0], V[0]))

72
126


In [1231]:
def optimal_nash_prod_multires(agents, resources, utility, p_utility):
    num_a = len(agents)
    num_r = len(resources)
    total_res = np.array(resources)[:,1]
    avail_res = total_res

    allocs = np.zeros((num_a, num_r))
    flallocs = allocs.flatten()
    
    opt_allocs = allocs
    opt_payoff = 0
    
    counter = 0
    while not np.array_equal(allocs[-1], total_res):
        i = 0
        carry = True
        while i < len(flallocs) and carry:
            carry = False
            flallocs[i] += 1
            if avail_res[i%len(avail_res)] == 0:
                carry = True
                flallocs[i] = 0
            i += 1
            allocs = flallocs.reshape((num_a, num_r))
            avail_res = total_res - allocs.sum(axis=0)
        payoff = 1
        for j in range(len(allocs)) :
            payoff *= complementing_utility(allocs[j], utility[j], p_utility[j])
        if opt_payoff < payoff :
            opt_payoff = payoff
            opt_allocs = np.copy(allocs)
        # print(allocs)
        # print(avail_res)
        counter += 1
    print(counter)
    # print(opt_payoff)
    return opt_allocs

In [1233]:
onpm = optimal_nash_prod_multires(N, R, U, V)

59


<hr>
#### Envy and envy up to one good calculations
<hr>

In [1208]:
def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
    return envy_map

In [1209]:
def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

In [1210]:
def envyMap_upto2(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                        if envy_map[i][j] < 0 :
                            envy_res_drop2_j = np.zeros((size_res, size_res))
                            max_pos_envy_drop2 = - maxsize
                            for p in range(len(envy_res_drop2_j)):
                                for q in range(len(envy_res_drop2_j[p])):
                                    if p == q:
                                        continue
                                    new_alloc_j_drop2 = list(allocs[j])
                                    new_alloc_j_drop2[p] = 0
                                    new_alloc_j_drop2[q] = 0
                                    envy_res_drop2_j[p][q] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                                    - complementing_utility(new_alloc_j_drop2, utilities[i], p_utilities[i])
                                    if envy_res_drop2_j[p][q] >= max_pos_envy_drop2:
                                        envy_map[i][j] = envy_res_drop2_j[p][q]
                                        max_pos_envy_drop2 = envy_res_drop2_j[p][q]
                
    return envy_map

In [1211]:
print(U)
print(V)

[[32, 71], [80, 26]]
[[25], [83]]


Envy map represents the envy between two agents. $envymap_{ij}$ is the amount with which i envies j

In [1212]:
rand_a = rand_allocs(R, N)
print("Random allocations : ", rand_a)
print(envyMap(rand_a, U, V))
print(envyMap_upto1(rand_a, U, V))

Random allocations :  [[2, 0], [1, 3]]
[[   0. -206.]
 [  81.    0.]]
[[ 0. 32.]
 [81.  0.]]


In [1235]:
print("Nash product allocations : ", onpm)
print(envyMap(onpm, U, V))
print(envyMap_upto1(onpm, U, V))
print(envyMap_upto2(onpm, U, V))

Nash product allocations :  [[1. 2.]
 [1. 1.]]
[[ 0. 25.]
 [-1.  0.]]
[[  0.  25.]
 [109.   0.]]
[[  0.  25.]
 [109.   0.]]


<hr>
#### Envy free experiments on large sample
<hr>

m => Number of resources <br>
R => Resources <id, value> <br>
n => Number of agents <br>
N => Agents <id, value, priority> <br>
U => Additive utilities
V => Complementing Utilities

Additive utilities

In [1214]:
np.random.seed(10)

ef1_tracker = []

for x in range(1000):
    m = 10
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 0
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    # print("Allocation : ", random_alloc)
    # print(envymap1)
    # print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

EF1 percent :  44.2


Complementing utilities

In [20]:
np.random.seed(10)

ef1_tracker = []

for x in range(1000):
    m = 10
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 100
        V.append(v_ijjp1)

    random_alloc = rand_allocs(R, N)
    envymap1 = envyMap_upto1(random_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    # print("Allocation : ", random_alloc)
    # print(envymap1)
    # print("Is EF1 allocation? ", ef1_tracker[x])
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

EF1 percent :  39.5


In [118]:
np.random.seed(10)

ef1_tracker = []

for x in range(100):
    m = 10
    R = [(i, np.random.randint(0, 100)) for i in range(m)]

    n = 2
    N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

    U = []
    for index, i in enumerate(N):
        u_i = np.random.random_sample(len(R)) * 100
        U.append(u_i)

    V = []
    for index, i in enumerate(N):
        v_ijjp1 = np.random.random_sample(len(R)//2) * 0
        V.append(v_ijjp1)

    nashp_alloc = optimal_nash_prod_wcomp(U, V)
    envymap1 = envyMap_upto1(nashp_alloc, U, V)
    ef1_tracker.append(not bool(np.sum(envymap1<0)))
    
    # print("Allocation : ", random_alloc)
    # print(envymap1)
    # print("Is EF1 allocation? ", ef1_tracker[x])
    print(x, " EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)
    
print("EF1 percent : ", np.sum(ef1_tracker)/len(ef1_tracker) * 100)

0  EF1 percent :  100.0
1  EF1 percent :  100.0
2  EF1 percent :  100.0
3  EF1 percent :  100.0
4  EF1 percent :  100.0
5  EF1 percent :  100.0
6  EF1 percent :  100.0
7  EF1 percent :  100.0
8  EF1 percent :  100.0
9  EF1 percent :  100.0
10  EF1 percent :  100.0
11  EF1 percent :  100.0
12  EF1 percent :  100.0
13  EF1 percent :  100.0
14  EF1 percent :  100.0
15  EF1 percent :  100.0
16  EF1 percent :  100.0
17  EF1 percent :  100.0
18  EF1 percent :  100.0
19  EF1 percent :  100.0
20  EF1 percent :  100.0
21  EF1 percent :  100.0
22  EF1 percent :  100.0
23  EF1 percent :  100.0
24  EF1 percent :  100.0
25  EF1 percent :  100.0
26  EF1 percent :  100.0
27  EF1 percent :  100.0
28  EF1 percent :  100.0
29  EF1 percent :  100.0
30  EF1 percent :  100.0
31  EF1 percent :  100.0
32  EF1 percent :  100.0
33  EF1 percent :  100.0
34  EF1 percent :  100.0
35  EF1 percent :  100.0
36  EF1 percent :  100.0
37  EF1 percent :  100.0
38  EF1 percent :  100.0
39  EF1 percent :  100.0
40  EF1 pe

<hr>
#### Store experimental data
<hr>

In [47]:
def writeNashAlloc(U, V, na, seed):
    filename = "nash_allocs_random_uv_" + str(seed)
    file = open(filename, "a")
    file.write("%s %s \t # U Shape\n" %(len(U), len(U[0])))
    for i in range(len(U)):
        file.write(" ".join(str(x) for x in U[i]))
        file.write("\n")
    
    file.write("%s %s \t # V Shape\n"  %(len(V), len(V[0])))
    for i in range(len(V)):
        file.write(" ".join(str(x) for x in V[i]))
        file.write("\n")
    
    file.write("%s %s \t # Alloc Shape\n"  %(len(na), len(na[0])))
    for i in range(len(na)):
        file.write(" ".join(str(x) for x in na[i]))
        file.write("\n")
        
    file.write("\n")
    file.close()

In [48]:
random_seed = 10

ef1_tracker = []
for s in range(11, 20):
    np.random.seed(s)
    
    for x in range(100):
        m = 10
        R = [(i, np.random.randint(0, 100)) for i in range(m)]

        n = 2
        N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]

        U = []
        for index, i in enumerate(N):
            u_i = np.random.random_sample(len(R)) * 100
            U.append(u_i)

        V = []
        for index, i in enumerate(N):
            v_ijjp1 = np.random.random_sample(len(R)//2) * 0
            V.append(v_ijjp1)

        nashp_alloc = optimal_nash_prod_wcomp(U, V)

        writeNashAlloc(U, V, nashp_alloc, random_seed)

In [1236]:
xt = (5, 4, 6)
len(xt)

3